In [2]:
import cv2 as cv
import numpy as np
import tensorflow as tf

from helpers.utils import show_image, normalize, preprocess_image, detect_sudoku, extract_cells

In [14]:
def isDelimiter(line, thinLine, boldLine):
    sum = line.sum()
    if thinLine - sum > sum - boldLine:
        return True
    return False

def extract_bold_lines(img):
    cell_width = 500 / 9  # each cell takes around 55.5 px
    padding = 5  # px

    lines_coordinates = []
    for i in range(9):
        Ya = i * cell_width
        Yb = (i+1) * cell_width
        Xa, Xb = 0, 0
        for j in range(8):
            Xa += cell_width
            Xb += cell_width
            line = [(int(Xa), int(Ya)), (int(Xa), int(Yb))]
            lines_coordinates.append(line)

    # now we have the coordinates of each small line

    # since each image will have some variation, we need to 
    # find the pixel sum of the boldest line
    # and the pixel sum of the thinnest line
    # and then compare each new line to the 2 sums
    # if the sum is closest to the sum of the boldest line => it's bold
    # otherwise it's thin
    boldLine = np.inf
    thinLine = 0

    # iterate through all 144 lines (72 horizontal and 72 vertical)
    # and compute the px sum of the boldest line and the px sum of the 
    # thinnest line
    for i in range(9):
        for j in range(8):
            line = lines_coordinates[i * 8 + j]
            Xa, Ya = line[0]
            Xb, Yb = line[1]
            vertical_line = img[Ya: Yb, Xa - padding: Xa + padding].copy()
            horizontal_line = img[Xa - padding: Xa + padding, Ya: Yb].copy()
            v_sum = vertical_line.sum()
            h_sum = horizontal_line.sum() 
            boldLine = min(v_sum, h_sum, boldLine)
            thinLine = max(v_sum, h_sum, thinLine) 
    # print(boldLine)
    # print(thinLine)


    # now iterate through the lines, check if it's bold or thin
    # and store the answer into the following 2 matrices
    vertical_lines_matrix = np.zeros((9, 8))
    horizontal_lines_matrix = np.zeros((8, 9))
    for i in range(9):
        for j in range(8):
            line = lines_coordinates[i * 8 + j]
            Xa, Ya = line[0]
            Xb, Yb = line[1]
            vertical_line = img[Ya: Yb, Xa - padding: Xa + padding].copy()
            if (isDelimiter(vertical_line, thinLine, boldLine)):
                vertical_lines_matrix[i][j] = 1
            # show_image("", vertical_line)
            horizontal_line = img[Xa - padding: Xa + padding, Ya: Yb].copy()
            if (isDelimiter(horizontal_line, thinLine, boldLine)):
                horizontal_lines_matrix[j][i] = 1
            # show_image("", horizontal_line)

    # print(vertical_lines_matrix)
    # print(horizontal_lines_matrix)
    return vertical_lines_matrix, horizontal_lines_matrix

In [4]:
from collections import deque as queue


def possible_move(old, new, horizontal_lines, vertical_lines):
    old_i, old_j = old
    new_i, new_j = new

    # first check if the move is in bounds
    if new_i < 0 or new_i > 8 or new_j < 0 or new_j > 8:
        return False

    # check if there is a line between the two cells:
    if old_i == new_i:  # horizontal move => vertical lines
        return not(vertical_lines[new_i][min(old_j, new_j)])
    else:  # vertical move => horizontal lines
        return not(horizontal_lines[min(old_i, new_i)][new_j])


def fill_region(start_i, start_j, region_number, regions, visited, h_lines, v_lines):
    # performs a bfs starting from (start_i, start_j) and
    # fills the reached cells with the value of region_number
    q = queue()
    q.append((start_i, start_j))
    visited[start_i][start_j] = 1
    while len(q):
        cell = q.popleft()
        x, y = cell
        regions[x][y] = region_number

        moves = [[-1, 0], [1, 0], [0, -1], [0, 1]]
        for move in moves:
            new_i = x + move[0]
            new_j = y + move[1]
            if possible_move((x, y), (new_i, new_j), h_lines, v_lines) and visited[new_i][new_j] == 0:
                q.append((new_i, new_j))
                visited[new_i][new_j] = 1


def fill_regions(regions, visited, h_lines, v_lines):
    # finds all the regions by performing a bfs starting from each
    # unvisited cell
    region_number = 1
    for i in range(9):
        for j in range(9):
            if not(visited[i][j]):
                fill_region(i, j, region_number, regions,
                            visited, h_lines, v_lines)
                region_number += 1


def extract_regions(img):
    vertical_lines, horizontal_lines = extract_bold_lines(img)

    # print(vertical_lines)
    # print(horizontal_lines)

    regions = np.full((9, 9), -1)
    visited = np.zeros((9, 9))

    fill_regions(regions, visited, horizontal_lines, vertical_lines)

    return regions

In [5]:
def extract_cells(img):
    lines_vertical = []
    lines_horizontal = []
    for i in range(0, 500, 55):
        lines_vertical.append([(i, 0), (i, 499)])
        lines_horizontal.append([(0, i), (499, i)])
    cells = []
    padding = 10 # padding each cell with 5px to make sure we don't crop out the lines too
    for i in range(len(lines_horizontal) - 1):
        for j in range(len(lines_vertical) - 1):
            y_min = lines_vertical[j][0][0] + padding
            y_max = lines_vertical[j + 1][1][0] - padding
            x_min = lines_horizontal[i][0][1] + padding
            x_max = lines_horizontal[i + 1][1][1] - padding
            cell = img[x_min:x_max, y_min:y_max].copy()
            cells.append(cv.resize(cell, (28, 28)))  # resizing to 28x28 to match the cnn input size
    return np.array(cells)

In [6]:
def extract_information(img):
    img = detect_sudoku(img)  # varying size, colored
    img = cv.resize(img, (500, 500))  # fixed size
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)  # fixed size, grayscale
    img = normalize(img)
    regions = extract_regions(img)
    # print(regions)
    # [[1 2 2 2 2 2 2 2 2]
    #  [1 1 1 3 3 3 2 4 4]
    #  [1 3 3 3 5 5 5 4 4]
    #  [1 1 6 3 3 3 5 4 4]
    #  [1 1 6 6 6 5 5 4 4]
    #  [6 6 6 6 7 5 5 8 4]
    #  [9 9 7 6 7 7 5 8 8]
    #  [9 9 7 7 7 7 8 8 8]
    #  [9 9 9 9 9 7 8 8 8]]

    model = tf.keras.models.load_model('saved_model/model.h5')
    cells = extract_cells(img)  # each cell is a 28x28 grayscale image
    cells = cells.reshape((cells.shape[0], 28, 28, 1)).astype('float32')
    predictions = model.predict(cells)  # make predictions
    labels = [np.argmax(prediction) for prediction in predictions]  # extract labels

    return [regions, labels]


In [7]:
def get_results(input_dir, output_dir, number_of_samples):
    for i in range(1, number_of_samples + 1):
        if i < 10:
            img = cv.imread(
                f"{input_dir}/0{i}.jpg")
        else:
            img = cv.imread(
                f"{input_dir}/{i}.jpg")
        img = cv.resize(img,(0,0),fx=0.2,fy=0.2)
        regions, labels = extract_information(img)
        cell_labels = np.array(labels)
        cell_labels = np.reshape(cell_labels, (9, 9))
        
        file = open(f'{output_dir}/jigsaw/{i}_predicted.txt', 'w')
        for j in range(9):
            for k in range(9):
                if cell_labels[j][k] == 0:
                    char = 'o'
                else:
                    char = 'x'
                file.write(str(regions[j][k]))
                file.write(char)
            if j != 8:
                file.write('\n')
        file.close()

        file = open(f'{output_dir}/jigsaw/{i}_bonus_predicted.txt', 'w')
        for j in range(9):
            for k in range(9):
                char = str(cell_labels[j][k])
                if char == '0':
                    char = 'o'
                file.write(str(regions[j][k]))
                file.write(char)
            if j != 8:
                file.write('\n')
        file.close()


In [16]:
input_dir = 'datasets/antrenare/jigsaw'
output_dir = 'results'
number_of_samples = 40  # number of input images
get_results(input_dir, output_dir, number_of_samples)